In [ ]:
# Step 1. Run this in Anaconda Command Shell before first-time use: conda install glpk
# Step 2. Run this in Jupyter before first-time use: !pip install pyomo

#### Import libraries

In [1]:
import yfinance as yf
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

#### Import stock data

In [2]:
# Stock tickers
tickers = ["UUP", "FXE", "FXY", "FXB"]

In [3]:
# Retrieve most recent closing stock prices
all_close_prices = []

for t in tickers:
    close_price = yf.download(t, period="1d")["Close"].iloc[0,0].astype(int)
    all_close_prices.append(close_price)

all_close_prices

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[28, 100, 62, 123]

In [4]:
close_price_with_symbol = dict(zip(tickers, all_close_prices))
close_price_with_symbol

{'UUP': 28, 'FXE': 100, 'FXY': 62, 'FXB': 123}

#### Model

In [5]:
model = pyo.ConcreteModel()

#### Sets

In [6]:
model.i = pyo.Set(initialize=tickers)

#### Parameters

In [7]:
model.C = pyo.Param(model.i, initialize=close_price_with_symbol)
C = model.C

#### Decision Variables

In [8]:
model.x = pyo.Var(model.i, within=pyo.NonNegativeIntegers)
x = model.x

#### Formula for Objective Function

In [9]:
obj_formula = sum(C[i]*x[i] for i in model.i)

#### Objective Function

In [10]:
def Objective_rule(model,i):
    return obj_formula
model.Objf = pyo.Objective(rule=Objective_rule, sense=pyo.maximize)

#### Constraints

In [11]:
def Constraint1(model):
    return x["UUP"] + x["FXE"] + x["FXY"] + x["FXB"] >= 4
model.Const1 = pyo.Constraint(rule=Constraint1)

def Constraint2(model):
    return x["FXE"] + x["FXB"] == 1
model.Const2 = pyo.Constraint(rule=Constraint2)

def Constraint3(model):
    return x["UUP"] == 3
model.Const3 = pyo.Constraint(rule=Constraint3)

def Constraint4(model):
    return obj_formula <= 275
model.Const4 = pyo.Constraint(rule=Constraint4)

#### Solve

In [12]:
Solver = SolverFactory("glpk")
results = Solver.solve(model)

#### Results

In [13]:
print(results)
print("Objective Function =", model.Objf())
for i in model.i:
    print("Number of", i, "to purchase =", x[i]())


Problem: 
- Name: unknown
  Lower bound: 269.0
  Upper bound: 269.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.04726290702819824
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective Function = 269.0
Number of UUP to purchase = 3.0
Number of FXE to purchase = 0.0
Number of FXY to purchase = 1.0
Number of FXB to purchase = 1.0
